In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import dateutil
import re

In [ ]:
boursorama_folder = '/home/aweng/afs/pybd/boursorama'

In [ ]:
compA_files = glob.glob(boursorama_folder + '/2019/compA 2019-*')
len(compA_files)

## Loading files

In [ ]:
%%time
compA = {}
for file in compA_files:
    date = dateutil.parser.parse('.'.join(' '.join(file.split()[1:]).split(".")[:-1]))
    compA[date] = pd.read_pickle(file)
compA = pd.concat(compA)
compA.sort_index(inplace=True) # chronological order
compA.head()

## Prepare Dataframe

last should be numerical but isn't, the other columns have a correct type.

In [ ]:
compA.isna().sum(), compA.info()

In [ ]:
def floatify(x) -> float:
    """
    Convert a string to a float, removing spaces if necessary.
    Used because .str.replace(' ', '').astype(float) + pd.to_numeric worked only on strings.
    Doing the operation on a float would result in a NaN.

    Handle:
    - regular numeric (13, 0.14, 2.343)
    - string ('13.0', '1321.491823', '12  222.222', '34.23 (c)')

    :param x: str|float
    :return: float
    """
    try:
        return float(re.sub(r'[^0-9.]', '', x))
    except:
        return x

In [ ]:
assert floatify('13') == 13, f"Expected 13, got {floatify('13')} with type {type(floatify('13'))}"
assert floatify('13.0') == 13, f"Expected 13, got {floatify('13.0')} with type {type(floatify('13.0'))}"
assert floatify('13.2 (c)') == 13.2, f"Expected 13.2, got {floatify('13.2 (c)')} with type {type(floatify('13.2 (c)'))}"
assert floatify(13.0) == 13, f"Expected 13, got {floatify(13.0)} with type {type(floatify(13.0))}"

In [ ]:
%%time
compA['last'] = compA['last'].apply(floatify).astype(float)
compA.isna().sum()

In [ ]:
axa_symbol = '1rPCS'
compA.loc[compA['symbol'] == axa_symbol].plot(y='volume', title=axa_symbol)

## Compute volume at time T instead of cumulative volume of the day

The volumes in the dataframe are intra-day total volume. Each day it gets reset to 0.
Having the volume as the volume traded since last timestamp would be more interesting.

In [ ]:
compA['volume_diff'] = compA.groupby([compA.index.get_level_values('symbol'), compA.index.get_level_values(0).date])['volume'].diff()
compA.fillna({'volume_diff': compA.volume}, inplace=True)
compA.head()

In [ ]:
compA.loc[compA['symbol'] == axa_symbol].plot(y='volume_diff', title=axa_symbol)

Some value are negative. Should'nt be. Let's remove them.\
Let's check that the volume gets reset correctly on each day while we are at it.

In [ ]:
# from 2019-01-03 17:35:00 to 2019-01-04 9:20:00
compA.loc[compA.symbol == axa_symbol].loc['2019-01-03 17:35:00':'2019-01-04 9:20:00']

In [ ]:
compA = compA[compA['volume_diff'] >= 0]
compA.shape

In [ ]:
compA.loc[compA['symbol'] == axa_symbol].plot(y='volume_diff', title="No more shit volume value")

Do we have fucked up volume after removing them? Lets redo the diff to see.

In [ ]:
compA['volume_diff'] = compA.groupby([compA.index.get_level_values('symbol'), compA.index.get_level_values(0).date])['volume'].diff()
compA.fillna({'volume_diff': compA.volume}, inplace=True)
compA.loc[compA.volume_diff < 0]

Oh no shit values again!

In [ ]:
nb_bad_values = len(compA.loc[compA.volume_diff < 0])
while nb_bad_values != 0:
    print(nb_bad_values)

    compA['volume_diff'] = compA.groupby([compA.index.get_level_values('symbol'), compA.index.get_level_values(0).date])['volume'].diff()
    compA.fillna({'volume_diff': compA.volume}, inplace=True)
    compA = compA[compA['volume_diff'] >= 0]

    nb_bad_values = len(compA.loc[compA.volume_diff < 0])

In [ ]:
compA.loc[compA['symbol'] == '1rPMC'].plot(y='volume_diff', title="No more shit volume value")

In [ ]:
compA.isna().sum()

# Day open/close/high/low

In [ ]:
grouped = compA.groupby([compA.index.get_level_values('symbol'), compA.index.get_level_values(0).date])
ohlc = grouped['last'].ohlc()
ohlc

Lets drop the NaN

In [ ]:
ohlc = ohlc.dropna()
ohlc.tail()

Lets that the values are correct for AXA on 2019-01-03

In [ ]:
ohlc.loc[axa_symbol].head()
# 18.664 18.928	18.646	18.650

In [ ]:
axa_20190103 = compA.loc[compA.symbol == axa_symbol].loc['2019-01-03']['last'].to_numpy()
print("Expected: 18.664, 18.928, 18.646,18.650")
"Acutal: ", axa_20190103[0], axa_20190103.max(), axa_20190103.min(), axa_20190103[-1]


# Lets plot candlestick (and bollingers bands)

In [ ]:
import plotly.graph_objects as go

In [ ]:
def candlestick(symbol: str) -> go.Candlestick:
    stock = ohlc.loc[symbol]
    ret = go.Candlestick(
        x=stock.index,
        open=stock['open'],
        high=stock['high'],
        low=stock['low'],
        close=stock['close'],
        text=symbol,
        name=symbol,
    )
    return ret

In [ ]:
symbols = [axa_symbol, '1rPVIE']

In [ ]:
stock = ohlc.loc['1rPCS']
# bollinger bands
stock['MA20'] = stock.close.rolling(window=20).mean()
stock['STD20'] = stock.close.rolling(window=20).std()
stock['upper'] = stock['MA20'] + (stock['STD20'] * 2)
stock['lower'] = stock['MA20'] - (stock['STD20'] * 2)

In [ ]:
fig = go.Figure(
    data=[candlestick(symbol) for symbol in symbols] + [
        go.Line(x=stock.index, y=stock['MA20'], name='MA20'),
        go.Scatter(x=stock.index, y=stock['upper'], name='upper bollinger', fill='tonexty', line_color='lightblue', opacity=0.3),
        go.Scatter(x=stock.index, y=stock['lower'], name='lower bollinger', fill='tonexty', line_color='lightblue', opacity=0.3),
    ])

fig.show()